<a href="https://colab.research.google.com/github/drouvotlucas/Mathematical-modelling/blob/main/pasm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#primal-dual programmation non linéaires

In [1]:
import numpy as np

##Fonctions

In [9]:
def pasm(Q,c,b,maxit,x0, eps=1):
  #eps vaut 1 pour une contrainte x<=b ou -1 pour une contrainte x>=b
  k=0
  xk=x0

  muk=np.zeros(c.shape)
  cond = True

  while(k<maxit and cond):
    Ak=[]
    Ik=[]

    for i in range(len(b)):
      if((eps*xk[i] > b[i]) or ((eps*xk[i] == b[i]) and (muk[i] >= 0))):
        Ak.append(i)
      if((eps*xk[i] < b[i]) or ((eps*xk[i] == b[i]) and (muk[i] < 0))):
        Ik.append(i)


    for i in Ak:
      xk[i] = b[i]

    M = Q[:,Ik][Ik,:]
    # Initialize sol to an empty array if len(Ik) == 0
    if len(Ik) != 0:
      if len(Ak) == 0:
        v = -c[Ik]
      else:
        v = -c[Ik] - np.dot(Q[:,Ak][Ik,:],xk[Ak])
      sol = np.linalg.solve(M,v)
    else:
      sol = np.array([]) # Or any other appropriate value
    print(sol)
    for i in range(len(Ik)):
      xk[Ik[i]] = sol[i]

    for i in range(len(Ik)):
      muk[Ik[i]] = -c[Ik[i]] -np.dot(Q[Ik[i]],xk)

    cond = False
    for i in range(len(b)):
      if (eps*(xk)[i] > b[i]):
        cond = True


    for i in Ak:
      if (muk[i] < 0):
        cond = True



    k +=1

  return xk, k


In [13]:
def pasmA(Q, c, Ai, b, maxit):
  Q2 = Ai@np.linalg.inv(Q)@np.transpose(Ai)
  c2 = (-1*np.transpose(c)@np.linalg.inv(Q)@np.transpose(Ai))[0]

  lamb0 = np.zeros(Ai.shape[0])
  lamb,k = pasm(Q2, c2, np.zeros(Ai.shape[0]), maxit, lamb0, -1)
  print("Q-1 = ", np.linalg.inv(Q))
  print("Atl+c = ", np.transpose(Ai)@lamb+c)
  print("c = ", c)
  # The result of np.transpose(Ai)@lamb is 1D, so you can't transpose along axis 1
  # Instead, we can just print it directly or reshape it to 2D before transposing
  print("Atl = ", (np.transpose(Ai)@lamb)) # or print("Atl = ", (np.transpose(Ai)@lamb).reshape(-1, 1)) if you need a column vector
  x = -np.linalg.inv(Q)@(np.transpose(Ai)@lamb+c)

  return x, k

##Problème basique

In [4]:
## Exemple simple avec les contraintes sous la forme x<=b  OK

Q = np.array([
[  2,    -1,     0],
[ -1,     2,    -1],
[  0,    -1,     2]])

c= np.array([
  -1,
  -2,
  -1])

b = np.array([
2.5000,
2.5000,
2.5000])


pasm(Q, c, b, 100, x0 = np.zeros(c.shape[0]))



[2. 3. 2.]
[1.75 1.75]


(array([1.75, 2.5 , 1.75]), 2)

In [5]:
## Exemple avec les contraintes sous la forme x <= b   OK

Q= np.array([
[  4,    -1,    0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[ -1,     4,   -1,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[  0,    -1,    4,    -1,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[  0,     0,   -1,     4,    -1,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[  0,     0,    0,    -1,     4,     0,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[ -1,     0,    0,     0,     0,     4,    -1,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[  0,    -1,    0,     0,     0,    -1,     4,    -1,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[  0,     0,   -1,     0,     0,     0,    -1,     4,    -1,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[  0,     0,    0,    -1,     0,     0,     0,    -1,     4,    -1,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[  0,     0,    0,     0,    -1,     0,     0,     0,    -1,     4,     0,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[  0,     0,    0,     0,     0,    -1,     0,     0,     0,     0,     4,    -1,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0,     0],
[  0,     0,    0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4,    -1,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0,     0],
[  0,     0,    0,     0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4,    -1,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0,     0],
[  0,     0,    0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4,    -1,     0,     0,     0,    -1,     0,     0,     0,     0,     0,     0],
[  0,     0,    0,     0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4,     0,     0,     0,     0,    -1,     0,     0,     0,     0,     0],
[  0,     0,    0,     0,     0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,     0,     4,    -1,     0,     0,     0,    -1,     0,     0,     0,     0],
[  0,     0,    0,     0,     0,     0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4,    -1,     0,     0,     0,    -1,     0,     0,     0],
[  0,     0,    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4,    -1,     0,     0,     0,    -1,     0,     0],
[  0,     0,    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4,    -1,     0,     0,     0,    -1,     0],
[  0,     0,    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4,     0,     0,     0,     0,    -1],
[  0,     0,    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,     0,     4,    -1,     0,     0,     0],
[  0,     0,    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4,    -1,     0,     0],
[  0,     0,    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4,    -1,     0],
[  0,     0,    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4,    -1],
[  0,     0,    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0,    -1,     0,     0,     0,    -1,     4]])
c=np.array([
   8.147236863931790,
   9.057919370756192,
   1.269868162935061,
   9.133758561390193,
   6.323592462254095,
   0.975404049994095,
   2.784982188670484,
   5.468815192049838,
   9.575068354342976,
   9.648885351992766,
   1.576130816775483,
   9.705927817606156,
   9.571669482429456,
   4.853756487228412,
   8.002804688888002,
   1.418863386272153,
   4.217612826262750,
   9.157355251890671,
   7.922073295595544,
   9.594924263929030,
   6.557406991565868,
   0.357116785741896,
   8.491293058687772,
   9.339932477575505,
   6.787351548577734])


b = -6 * np.ones(c.shape)

pasm(Q, c, b, 5, x0 = -10*np.ones(c.shape[0]))

[ -5.92131041  -8.95657716  -8.80931372 -10.05318458  -6.81120282
  -6.58142762 -12.03776515 -14.95762497 -15.4584632  -10.86803426
  -7.39123087 -14.87044867 -18.0561426  -16.37994065 -11.55358565
  -6.53691636 -12.29072824 -16.44488664 -15.59781467 -10.963563
  -5.04684295  -7.09304844 -10.67750581 -10.68079509  -7.10792741]
[ -9.00432672  -8.84195275 -10.07379429  -6.82111624  -6.65467025
 -12.11743477 -15.01982181 -15.49834963 -10.88707819  -7.52584218
 -15.0059381  -18.15273492 -16.43763586 -11.57996153  -6.86662957
 -12.5218127  -16.57587443 -15.66574088 -10.99232739  -7.42119586
 -10.80585398 -10.73505255  -7.12868287]
[ -8.98356267  -8.81979739 -10.05758169  -6.81287454  -6.60841313
 -12.05653394 -14.96817703 -15.46389627 -10.870324    -7.40171454
 -14.88100073 -18.06366531 -16.38443402 -11.55563983  -6.54131347
 -12.2961613  -16.44938001 -15.60077816 -10.96499662  -5.04851466
  -7.09533819 -10.67955999 -10.68222871  -7.10864422]
[ -9.00432672  -8.84195275 -10.07379429  -6.8211

(array([ -6.        ,  -8.98356267,  -8.81979739, -10.05758169,
         -6.81287454,  -6.60841313, -12.05653394, -14.96817703,
        -15.46389627, -10.870324  ,  -7.40171454, -14.88100073,
        -18.06366531, -16.38443402, -11.55563983,  -6.54131347,
        -12.2961613 , -16.44938001, -15.60077816, -10.96499662,
         -5.04851466,  -7.09533819, -10.67955999, -10.68222871,
         -7.10864422]),
 5)

##Problème Ax <= b donc transformation de x


In [14]:
Q = np.array([
[2.3546, 1.6361, 1.8427, 2.1537],
[1.6361, 1.6617, 1.5320, 1.4873],
[1.8427, 1.5320, 2.4314, 2.2958],
[2.1537, 1.4873, 2.2958, 2.8471]])

c=np.array([
[-0.7583],
[-0.2899],
[-1.0962],
[-1.2270]])

Ai = np.array([
[0.202700, 0.272100, 0.746700, 0.465900],
[0.198700, 0.198800, 0.445000, 0.418600],
[0.603700, 0.015200, 0.931800, 0.846200]])

b = np.array([
[0.5251],
[0.2026],
[0.6721]])

x0 = np.zeros(c.shape[0])

pasmA(Q, c, Ai, b, 100)



[]
Q-1 =  [[ 2.41571267 -1.56724149  0.45723946 -1.3773638 ]
 [-1.56724149  2.4577854  -1.1230082   0.80717431]
 [ 0.45723946 -1.1230082   2.28413155 -1.60107329]
 [-1.3773638   0.80717431 -1.60107329  2.26253455]]
Atl+c =  [[-0.7583 -0.7583 -0.7583 -0.7583]
 [-0.2899 -0.2899 -0.2899 -0.2899]
 [-1.0962 -1.0962 -1.0962 -1.0962]
 [-1.227  -1.227  -1.227  -1.227 ]]
c =  [[-0.7583]
 [-0.2899]
 [-1.0962]
 [-1.227 ]]
Atl =  [0. 0. 0. 0.]


(array([[ 0.18869213,  0.18869213,  0.18869213,  0.18869213],
        [-0.71656594, -0.71656594, -0.71656594, -0.71656594],
        [ 0.56051269,  0.56051269,  0.56051269,  0.56051269],
        [ 0.21057822,  0.21057822,  0.21057822,  0.21057822]]),
 1)

In [15]:
Ai = np.array([
    [-1, 2],
    [1, -2]
])

b = np.array([
    [2],
    [2]
])

x0 = np.array([
    [2],
    [0]
])

Q = np.array([
    [2, 0],
    [0, 2]
])

c = np.array([
    [-2],
    [-5]
])


pasmA(Q, c, Ai, b, 1000)

[]
Q-1 =  [[0.5 0. ]
 [0.  0.5]]
Atl+c =  [[-2. -2.]
 [-5. -5.]]
c =  [[-2]
 [-5]]
Atl =  [0. 0.]


(array([[1. , 1. ],
        [2.5, 2.5]]),
 1)